
<p><img src="https://oceanprotocol.com/static/media/banner-ocean-03@2x.b7272597.png" alt="drawing" width="800" align="center"/>


<h1><center>Ocean Protocol - Manta Ray project</center></h1>
<h3><center>Decentralized Data Science and Engineering, powered by Ocean Protocol</center></h3>
<p>Version 0.5.3 - beta</p>
<p>Package compatibility: squid-py v0.6.13, keeper-contracts 0.10.3, utilities 0.2.2,
<p>Component compatibility (Nile): Brizo v0.3.12, Aquarius v0.3.4, Nile testnet smart contracts 0.10.3</p>

<p><a href="https://github.com/oceanprotocol/mantaray">mantaray on Github</a></p>
<p>

# Pre-sail checklist - Ocean protocol component status
With simulated Kubernetes endpoints deployed, this script will make a simple
HTTP request to each, and report the status returned.


In [ ]:
# Standard imports
import requests
import os
import logging
from pathlib import Path

from mantaray_utilities import logging as manta_logging

In [ ]:
# For this test, set the configuration environment variable for kubernetes.
# here it is hard-coded for IPython execution, but in general, it is set in your system environment.
# manta_config.name_deployment_type()
# os.environ['USE_K8S_CLUSTER'] = 'true'

manta_logging.logger.setLevel('INFO')

# Get the configuration file path for this environment
OCEAN_CONFIG_PATH = Path(os.environ['OCEAN_CONFIG_PATH'])
assert OCEAN_CONFIG_PATH.exists()
logging.info("OCEAN_CONFIG_PATH:{}".format(OCEAN_CONFIG_PATH))

import configparser
config = configparser.ConfigParser()
config.read(OCEAN_CONFIG_PATH)

In [ ]:
# The endpoints (microservices) are defined in the below dictionary

In [ ]:
# For now, the endpoints are hard-coded by the dev-ops team.
endpoints_dict = {
    'aquarius': config['resources']['aquarius.url'],
    'brizo': config['resources']['brizo.url'],
    'Ethereum node': config['keeper-contracts']['keeper.url'],
    'secret_store': config['keeper-contracts']['secret_store.url'],
}

swagger_pages = dict()
swagger_pages['aquarius Swagger documentation'] = endpoints_dict['aquarius'] + '/api/v1/docs/'
swagger_pages['brizo Swagger documentation'] = endpoints_dict['brizo'] + '/api/v1/docs/'


def check_endpoint(endpoint_name, endpoint_url, verb='GET', ):
    """HTTP Request on the given URL"""
    res = requests.request(verb, endpoint_url)
    logging.debug("{} : returns {}".format(endpoint_name, res.status_code))
    return res

In [ ]:
# The microscervices for MetaData storage (aquarius) and for service negotiation (brizo) have Swagger documentation :)

In [ ]:
print("Aquarius MetaData storage API:", swagger_pages['aquarius Swagger documentation'])
print("Brizo Access API:", swagger_pages['brizo Swagger documentation'])

Finally, we will iterate over the endpoint and check for a response


In [ ]:
flag_fail = False
for endpoint in endpoints_dict:
    with manta_logging.LoggerCritical():
        print("Checking {} at {}".format(endpoint, endpoints_dict[endpoint]))
        try:
            res = check_endpoint(endpoint, endpoints_dict[endpoint])
            if 'Content-Type' in res.headers:
                if res.headers['Content-Type'] == 'application/json':
                    if 'software' in res.json().keys() and 'version' in res.json().keys():
                        print("\t Success: {} v{}".format(res.json()['software'], res.json()['version']))
                else:
                    print("\t Success")
            else:
                print("\t Success")
        except:
            flag_fail = True
            print('\t Failed!')

if flag_fail:
    print("Failure in a component, please contact an administrator on our Gitter channel - https://gitter.im/oceanprotocol/Lobby")